In [1]:

import csv
import random
from collections import defaultdict
from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import flatten
import pandas as pd
import numpy as np

music_data=pd.read_csv(r'E:\arshad\term 2\NLP\hw1\music.csv')
#music_data = music_data.drop('-')
Name_music=music_data['name']
Note_music=music_data['note']

Dastgah_music=music_data['dastgah']


#### مجموعه داده مورد نظر را دریافت میکنیم

## perplexity

In [2]:
import math
def calculate_perplexity(notes_list, model, n):
    perplexities = []
    for notes in notes_list:
        N = len(notes)
        log_prob_sum = 0
        notes=list(notes)
        notes = ["<s>"] * (n - 1) + notes + ["</s>"]
        for i in range(n, N + 1):
            context = tuple(notes[i-n:i-1])
            next_note = notes[i-1]
            if context in model and next_note in model[context]:
                prob = model[context][next_note]
            else:
                prob = 1 / (len(model[context]) + len(model))
            log_prob_sum += math.log2(prob)
        perplexity = 2 ** (-log_prob_sum / (N - n + 1))
        perplexities.append(perplexity)
    return perplexities


#### در این قطعه کد معیار سرگشتگی را برای هر قطعه کد محاسبه میکنیم. البته من برای این تمرین یک مجموعه را دریافت و یک لیست را بازگردانی میکنم که میزان سرگشتگی را بازگردانی میکند.


In [3]:
def get_notes_from_csv2(df):
    notes = []
    Note_music=df['note']
    for i in range(len(df)):
        sen=Note_music[i].split(' ')
        sen.remove('')
        for s in sen:
            notes.append(s)
    return notes

#### در این قطعه کد ما مجموعه داده را به لیستی از نوت ها تفکیک میکنیم

In [4]:
import re
def create_ngram_model(n, notes, alpha):
    
    model = defaultdict(list)
    for i in range(len(notes)-n):
        sequence = tuple(notes[i:i+n])
        next_note = notes[i+n]
        model[sequence].append(next_note)
    ## smoothing
    unique_sequences = set(model.keys())
    for sequence in unique_sequences:
        next_notes = model[sequence]
        total_count = len(next_notes)
        for note in set(next_notes):
            count = next_notes.count(note)
            probability = (count + alpha) / (total_count + len(set(notes)))
            model[sequence][model[sequence].index(note)] = (note, probability)
    return model

#### در این قطعه کد ما مدل انگراممان را میسازیم. ابتدا تاپل هایی با تعداد عدد ان را میسازیم. سپس فرایند هموارسازی را روی ان اعمال میکنیم

In [5]:
def generate_notes(model, start_sequence, length,n):
    current_sequence = tuple(start_sequence.split())
    x=current_sequence[len(current_sequence)-n:len(current_sequence)]
    x=tuple(x)
    output_notes = list(current_sequence)
    for i in range(length):
        x=current_sequence[len(current_sequence)-n:len(current_sequence)]
        x=tuple(x)
        if x in model:
            possible_next_notes = model[x]
            # Choose a random next note from the possible options
            next_note = possible_next_notes[random.randrange(len(possible_next_notes))]
            output_notes.append(next_note[0])
            # Update sequence with the new note
            current_sequence = current_sequence[1:] + (next_note[0],)
        else:
            break
    return ' '.join(output_notes)

#### در قطعه کد بالا انتهای رشته را میگیرد یعنی اگر بایگرام باشد اخرین نوت را در نظر میگیرد و اگر تریگرام باشد، دو نوت اخر را در نظر میگیرد. سپس به مدل رجوع میکند و با توجه به احتمالات ممکن یک نوت را به انتهای رشته اضافه میکند. این فرایند تا زمانی که طول رشته 14 شود ادامه می یابد.

# bigram

#### در قطعه کدهای زیر مدل بایگرام فراخوانی میشود و رشته ساخته می شود و سپس به دو لیست اضافه میشود . سپس ان دو لیست را به تابع محاسبه سرگشتگی میدهیم و این معیار را برای هر رشته محاسبه میکنیم.

In [6]:
notes=get_notes_from_csv2(music_data)
model1=create_ngram_model(1, notes, alpha=1)
model2=create_ngram_model(1, notes, alpha=0.5)
notes1=[]
notes2=[]

In [7]:
not1=generate_notes(model1, 'S S S S', 10,1)
not2=generate_notes(model2, 'S S S S', 10,1)
notes1.append(not1)
notes2.append(not2)
print(not1)
print(not2)

S S S S A S G S C Bb C S D E
S S S S G C D D D B C B S D


In [8]:
not1=generate_notes(model1, 'G C G C', 10,1)
not2=generate_notes(model2, 'G C G C', 10,1)
notes1.append(not1)
notes2.append(not2)
print(not1)
print(not2)

G C G C F G A B S G B B B C
G C G C C G A D S D D C C C


In [9]:
not1=generate_notes(model1, 'A B C D', 10,1)
not2=generate_notes(model2, 'A B C D', 10,1)
notes1.append(not1)
notes2.append(not2)
print(not1)
print(not2)

A B C D Bb S G G D E D E E E
A B C D D D D C C C C C C C


In [10]:
not1=generate_notes(model1, 'A S B Db',10,1)
not2=generate_notes(model2, 'A S B Db',10,1)
notes1.append(not1)
notes2.append(not2)
print(not1)
print(not2)

A S B Db
A S B Db


In [11]:
not1=generate_notes(model1, 'A Bb C D',10,1)
not2=generate_notes(model2, 'A Bb C D',10,1)
notes1.append(not1)
notes2.append(not2)
print(not1)
print(not2)

A Bb C D E E E E S D D D D F
A Bb C D D E S D E S B A A E


In [12]:
perplexity = calculate_perplexity(notes1, model1, 2)
print("Perplexity:", perplexity)

Perplexity: [24.64591563611437, 27.617008908077093, 27.725202619895747, 22.23574609159935, 27.89984514893912]


In [13]:
perplexity = calculate_perplexity(notes2, model2, 2)
print("Perplexity:", perplexity)

Perplexity: [25.471637089954047, 30.116395821065208, 29.525829674541768, 21.086221420048247, 26.226336788315436]


# trigram

#### در قطعه کدهای زیر مدل تریگرام فراخوانی میشود و رشته ساخته می شود و سپس به دو لیست اضافه میشود . سپس ان دو لیست را به تابع محاسبه سرگشتگی میدهیم و این معیار را برای هر رشته محاسبه میکنیم.

In [14]:
notes=get_notes_from_csv2(music_data)
model1=create_ngram_model(2, notes, alpha=1)
model2=create_ngram_model(2, notes, alpha=0.5)
notes1=[]
notes2=[]

In [15]:
not1=generate_notes(model1, 'S S S S', 10,2)
not2=generate_notes(model2, 'S S S S', 10,2)
notes1.append(not1)
notes2.append(not2)
print(not1)
print(not2)

S S S S G C S Ab G S D Bb A A
S S S S G C C C D E E D D Bb


In [16]:
not1=generate_notes(model1, 'G C G C', 10,2)
not2=generate_notes(model2, 'G C G C', 10,2)
notes1.append(not1)
notes2.append(not2)
print(not1)
print(not2)

G C G C C C C F G G F G F F
G C G C C C D D E F C C C C


In [17]:
not1=generate_notes(model1, 'A B C D', 10,2)
not2=generate_notes(model2, 'A B C D', 10,2)
notes1.append(not1)
notes2.append(not2)
print(not1)
print(not2)

A B C D E F G E D E E E E D
A B C D B


In [18]:
not1=generate_notes(model1, 'A S B Db', 10,2)
not2=generate_notes(model2, 'A S B Db', 10,2)
notes1.append(not1)
notes2.append(not2)
print(not1)
print(not2)

A S B Db
A S B Db


In [19]:
not1=generate_notes(model1, 'A Bb C D', 10,2)
not2=generate_notes(model2, 'A Bb C D', 10,2)
notes1.append(not1)
notes2.append(not2)
print(not1)
print(not2)

A Bb C D E C C Bb C S S G C S
A Bb C D D E F E E C C D Bb B


In [20]:
perplexity = calculate_perplexity(notes1, model1, 3)
print("Perplexity:", perplexity)

Perplexity: [84.86614213547846, 93.71466915887856, 98.94916990072934, 99.99999999999997, 100.00000000000023]


In [21]:
perplexity = calculate_perplexity(notes2, model2, 3)
print("Perplexity:", perplexity)

Perplexity: [84.23515654184608, 90.39604469775983, 94.55949265821245, 95.99999999999993, 97.66398855933213]


## unigram

In [22]:
from collections import Counter
import re
def create_unigram_model(df, alpha=1):
    tokens = []
    for i in range(len(df['note'])):
        x = re.findall(r'\b\w+\b', df['note'][i].lower())
        tokens.extend(x)
    token_counts = Counter(tokens)
    total_tokens = sum(token_counts.values())
    probabilities = {}
    vocab_size = len(token_counts.keys())
    for token, count in token_counts.items():
        probabilities[token] = (count + alpha) / (total_tokens + alpha * vocab_size)
    return probabilities

#### در کد بالا مدل یونیگرام را میسازیم و فرایند هموارسازی را با دریافت مقدار الفا بر روی ان تاثیر میدهیم.

In [23]:
import random
def get_sampled_ind(probs):
    cum_probs = [sum(probs[:i+1]) for i in range(len(probs))]
    if cum_probs[-1] >=1:
        x=(cum_probs[-1]-1)
        s=float(x)
        cum_probs=[i-s for i in cum_probs]
    else:
        x=(1-cum_probs[-1])
        s=float(x)
        cum_probs=[i+s for i in cum_probs]
    rand = random.random()
    for i in range(len(cum_probs)):
        if rand <= cum_probs[i]:
            index=i
            return index

#### تابع بالا یک ایندکس را به عنوان شماره بیت در مدل برای ما بازگردانی میکند

In [24]:
def generate_note_unigram(model, start_sequence_str, length):
    start_sequence = list(start_sequence_str.split())
    output_notes = start_sequence
    for i in range(length):
        rand_state=len(start_sequence)-1
        output_notes[rand_state].strip(' ')
        x=list(model.keys())
        probabilities = []
        #print(model)
        for value in model.values():
            probabilities.append(value)
        ind=get_sampled_ind(probabilities)
        output_notes.insert(rand_state+1,x[ind])
    output=' '.join(output_notes)
    return output

#### در این تابع نیز ما مانند مدل قبلی عمل میکند

In [25]:
def perplexity_unigram(text, unigram_model):
    perplexity=[]
    for item in text:
        tokens = item.split()
        log_prob_sum = 0
        for token in tokens:
            if token in unigram_model:
                log_prob_sum += math.log(unigram_model[token])
            else:          
                log_prob_sum += math.log(0.0001)
        
        log_perplexity = -1 / (len(tokens) * log_prob_sum)
        perplexity.append(2 ** log_perplexity)
    return perplexity

#### در قطعه کد بالا سرگشتگی را برای یک مدل یونیگرام محاسبه میکنیم.

#### در قطعه کدهای زیر مدل یونیگرام فراخوانی میشود و رشته ساخته می شود و سپس به دو لیست اضافه میشود . سپس ان دو لیست را به تابع محاسبه سرگشتگی میدهیم و این معیار را برای هر رشته محاسبه میکنیم.

In [26]:
df=pd.read_csv(r'E:\arshad\term 2\NLP\hw1\music.csv')
notes = get_notes_from_csv2(df)
model1 = create_unigram_model(df,0.5)
model2 = create_unigram_model(df,1)
notes1=[]
notes2=[]

In [27]:
not1=generate_note_unigram(model1, 'S S S S', 10)
not2=generate_note_unigram(model2, 'S S S S', 10)
notes1.append(not1)
notes2.append(not2)
print(not1)
print(not2)

S S S S f c c g bb a g c b c
S S S S a c g bb g c e c a b


In [28]:
not1=generate_note_unigram(model1, 'G C G C', 10)
not2=generate_note_unigram(model2, 'G C G C', 10)
notes1.append(not1)
notes2.append(not2)
print(not1)
print(not2)

G C G C g s f e s e bb c c g
G C G C b c e e g g a c bb d


In [29]:
not1=generate_note_unigram(model1, 'A B C D', 10)
not2=generate_note_unigram(model2, 'A B C D', 10)
notes1.append(not1)
notes2.append(not2)
print(not1)
print(not2)

A B C D bb bb ab a bb g e g g eb
A B C D g g g c g a g ab f c


In [30]:
not1=generate_note_unigram(model1, 'A Bb C D', 10)
not2=generate_note_unigram(model2, 'A Bb C D', 10)
notes1.append(not1)
notes2.append(not2)
print(not1)
print(not2)

A Bb C D g g bb c a d g d ab s
A Bb C D bb a g e e b bb g e f


In [31]:
not1=generate_note_unigram(model1, 'A S B Db', 10)
not2=generate_note_unigram(model2, 'A S B Db', 10)
notes1.append(not1)
notes2.append(not2)
print(not1)
print(not2)

A S B Db s bb e e a c b e d e
A S B Db a e b e c bb c a e f


In [32]:
perplexity = perplexity_unigram(notes1, model1)
print("Perplexity:", perplexity)

Perplexity: [1.0008562494516828, 1.0008459565234427, 1.0007897912863555, 1.0008334172698121, 1.000827976909278]


In [33]:
perplexity = perplexity_unigram(notes2, model2)
print("Perplexity:", perplexity)

Perplexity: [1.0008520309500284, 1.0008514535946609, 1.0008543257092892, 1.0008321080315923, 1.000832329758147]
